<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/DataScience/210125_Feature_Selection_%26_extension_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#210125 Feature Selection and Extension.

오늘부터는 특성 선택과 추출에 대해서 공부하고자 한다. 앞에서 접해보았던 머신러닝 기법들 전체적으로 가지고 있는 해결점에 대해 고려해보자.

## 0. 차원의 저주 및 특성 선택/압축 개요
만일 어떤 머신러닝 분석 문제에서 특성이 1개인 경우와, 특성이 1000개인 경우 두 가지가 있다고 보자. 특성이 1개인 경우에는 분석이 1개겠지만, 특성이 1000개인 경우에는 분석의 어려움도 있겠지만 서로간의 연관성에 의해 노이즈가 더 발생할 수 있다.

따라서 우리는 **중요한 특성만 선택**하던가, 특**성을 압축하여 효과적으로 분석할 길을 고려해야 할 것**이다.

대표적인 특성 선택과 추출에는 다음과 같은 방법들이 있다.

- 특성 선택
  * Sequential feature selection
  탐욕적 알고리즘 기반으로, 초기 특성 공간을 축소한다. 관계없는 특성이나 잡음을 제거해야 한다.
  전통적인 순차 특성 알고리즘은 순차 후진 선택(Sequential Backward Selection, SBS)를 취한다.

- 특성 압축
  * Principal Component Analysis(주성분 분석)
  * Linear Discriminant Analysis(선형판별분석)
  * Kernel Principal Component Analysis (커널PCA)
    -> 비선형 차원 축소에 사용 가능.

앞으로 이 방법들이 어떤 식으로 사용되는지, 그 원리와 예시를 좀 보려고 한다.

## 1. Sequential Backward Selection의 개요

순차 후진 선택은 모든 feature가 주어진 상황에서, 부분공간으로 특성을 축소시기키기 위해 하나씩 feature을 제거해 나가는 방법이다.

목표는 당연히, **모델 성능을 가장 적게 희생**하는 것이다.

가장 떠오르는 쉬운 방법은 **가장 많이 성능을 깨치는 것부터 하나씩 제거**해 나가는 것이다. 실제로 이를 greedy algorithm(탐욕적 알고리즘)의 예시로 볼 수 있는데, greedy algorithm이란 최적화나 탐색 문제의 각 단계에서 부분적으로 선택할 수 있는 최적의 선택을 하는 것이다.

이를 위해서는 기준 함수가 필요할 것인데, 이 기준함수는 **특성을 제거하기 전의 모델의 성능 차이**이다. 기준 함수가 최대화가 되는 특성을 하나씩 제거하다가 목표하는 특성 개수가 되면 종료한다.


이런, 아쉽게도 scikit-learn에는 해당 모델이 없어서 직접 코딩을 해야 한다.

참조 : https://vitalflux.com/sequential-backward-feature-selection-python-example/



## 2. Sequential Backward Selection code

코드가 쉽지는 않다. 주석을 달면서 하나하나 이해해보자.


In [2]:
from sklearn.metrics import accuracy_score
from itertools import combinations
from sklearn.base import clone
import numpy as np
 
class SequentialBackwardSearch():

    def __init__(self, estimator, k_features):
        self.estimator = clone(estimator)
        self.k_features = k_features
         
    def fit(self, X_train, X_test, y_train, y_test):  # 훈련셋/테스트셋
        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train.values, X_test.values, 
                                 y_train.values, y_test.values, self.indices_)
        self.scores_ = [score]
        
        # 원하는 개수가 돌 때까지 돌아야 하므로

        while dim > k_features:
            scores = []
            subsets = []

            # 존재하는 특성의 조합별로 모델을 훈련하고 그 점수를 기록한다.
            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train.values, X_test.values, y_train.values, y_test.values, p)
                scores.append(score)
                subsets.append(p)

            best_score_index = np.argmax(scores) # 최고점 위치 기록
            self.scores_.append(scores[best_score_index]) # 최고점 기록
            self.indices_ = subsets[best_score_index] # 최고점 특성 기록
            self.subsets_.append(self.indices_) # 특성의 위치 기록
            dim -= 1 # 하나 날렸으니까 차원 하나 줄여야함...
     
    # 데이터 셋 변환을 통해 필요 변수만 추린다.
    def transform(self, X):
        return X.values[:, self.indices_]
     
    # 최종 선택 기준으로 훈련한다.
    def _calc_score(self, X_train, X_test, y_train, y_test, indices):
        self.estimator.fit(X_train[:, indices], y_train.ravel())
        y_pred = self.estimator.predict(X_test[:, indices])
        score = accuracy_score(y_test, y_pred)
        return score

코드에서는 먼저, 목표 특성 개수를 k_features로 지정한다.

accuracy_score 함수로 특성 부분집합에 대한 모델 성능을 평가한다.
최적 조합 정확도 점수는 self.scores_ 리스트에 모은다. 이 점수를 사용하여 나중에 결과를 평가한다.
최종 선택된 특성의 열 인덱스는 self.indices_에 할당된다.

이를 바탕으로, transform에서 선택된 열로만 구성된 새로운 데이터를 반환한다.


## 3. PCA의 개요

다음은 주성분 분석으로 불리는 PCA(Principal Component Analysis)이다. PCA는 데이터간의 상관관계를 기반으로 특성을 잡아낸다.

- 분산이 가장 큰 방향을 찾는다.
- 차원이 좀 더 작거나 같은 새로운 부분공간으로 투영시킨다.
- 새로운 특성축은 기존의 특성축과 직교한다.

Orthogonal 하게 자른다는 점에서, 우리는 어떤 행렬을 생각하여 eigenvalue를 활용하여 행렬을 쪼개야 할 것임을 예측할 수 있다.

PCA에서는 차원을 축소하기 위해 변환 행렬을 만들어야 한다.
$d$차원에서 $k$차원으로 축소하기 위해, 우리는 $d \times k$ 행렬을 만들어야 한다. 행렬을 통해 벡터를 부분공간으로 mapping 시켜야한다.

몇 가지 주의사항과 관찰할 수 있는 부분을 알 수 있다.

- 변환 행렬을 만든다는 점에서, PCA의 방향들은 각 데이터의 범위에 민감하게 반응할 수 밖에 없다. **scaling 과정이 선행되어야만 한다**.
- 모든 주성분은 다른 주성분들과 **모두 orthogonal**해야 가장 큰 분산을 가진다. 즉 **"직교행렬"을 만들어야 한다.**


구체적인 알고리즘과 PCA와 관련된 수학적 배경은 내일 마저 공부하려고 한다.